In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
import re
import spacy
nlp = spacy.load('en')

Common Preprocessing Steps
http://pages.cs.wisc.edu/~jerryzhu/cs769/text_preprocessing.pdf
Preprocessing text is called tokenization or text normalization
1. Throw away unwanted stuff(e.g., HTML tags)
2. Word boundaries: white space and punctuations
3. Stemming (Lemmatization
4. Stopword removal
5. Capitalization, case folding

https://datascience.stackexchange.com/questions/11402/preprocessing-text-before-use-rnn/11421

ML Workflow: https://www.pewe.sk/datalys/wp-content/uploads/sites/3/2017/10/2017-10-05-Datalys-ML-Workflow.pdf

https://www.kdnuggets.com/tag/data-preprocessing
https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html
https://www.kdnuggets.com/2018/05/getting-started-spacy-natural-language-processing.html

Preprocess review comments for word2vac model
preprocess for sentiment analysis

create bigrams
https://stackoverflow.com/questions/46129335/get-bigrams-and-trigrams-in-word2vec-gensim

In [2]:
reviews = pd.read_json('data/reviews_old_play.json')

In [3]:
reviews.head(2)

,attendence,class_names,comments,date,for_credits,grades_revieved,level_of_difficulty,overall_quality,prof_id,rating_type,tags_by_user,textbook_used,thumbs_down,thumbs_up,would_take_again
0,Mandatory,SOC106,Clawson is really caring and interested in his...,2017-12-14,Yes,N/A,3,4.0,100001,good,"[CARING, RESPECTED, GET READY TO READ]",No,0,0,Yes
1,Mandatory,SOC353,This class was definitely not what I thought i...,2017-11-15,N/A,B,3,3.0,100001,average,"[CLEAR GRADING CRITERIA, PARTICIPATION MATTERS...",Yes,0,1,No


In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183186 entries, 0 to 183185
Data columns (total 15 columns):
attendence             183186 non-null object
class_names            183186 non-null object
comments               183186 non-null object
date                   183186 non-null datetime64[ns]
for_credits            183186 non-null object
grades_revieved        183186 non-null object
level_of_difficulty    183186 non-null int64
overall_quality        183186 non-null float64
prof_id                183186 non-null int64
rating_type            183186 non-null object
tags_by_user           183186 non-null object
textbook_used          183186 non-null object
thumbs_down            183186 non-null int64
thumbs_up              183186 non-null int64
would_take_again       183186 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(9)
memory usage: 22.4+ MB


In [4]:
sample_review = reviews.comments[10]
sample_review

"horrible class. DONT take it.  dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well. paper due every week, do all of them. and id write all the essays so that you can get a good grade. go to class because of random quizzes."

In [5]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 49.1 ms, sys: 3.83 ms, total: 53 ms
Wall time: 93.5 ms


In [6]:
parsed_review

horrible class. DONT take it.  dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well. paper due every week, do all of them. and id write all the essays so that you can get a good grade. go to class because of random quizzes.

In [7]:
for num, sent in enumerate(parsed_review.sents):
    print('Sentence {}:\n{}\n'.format(num+1, sent))

Sentence 1:
horrible class.

Sentence 2:
DONT take it.  

Sentence 3:
dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well.

Sentence 4:
paper due every week, do all of them.

Sentence 5:
and id write all the essays so that you can get a good grade.

Sentence 6:
go to class because of random quizzes.



In [27]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)

Entity 1: every week - DATE


In [10]:
df = pd.DataFrame(columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'alpha', 'stop'])
for token in parsed_review:
    token_info = [token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop]
    df2 = pd.DataFrame([token_info], columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'alpha', 'stop'])
    #df = pd.DataFrame(np.insert(df.values, 0, values=[token_info], axis=0))
    df = df.append(df2)
df

,text,lemma,pos,tag,dep,shape,alpha,stop
0,horrible,horrible,ADJ,JJ,amod,xxxx,True,False
0,class,class,NOUN,NN,ROOT,xxxx,True,False
0,.,.,PUNCT,.,punct,.,False,False
0,DONT,dont,NOUN,NNS,nsubj,XXXX,True,False
0,take,take,VERB,VBP,ROOT,xxxx,True,True
0,it,-PRON-,PRON,PRP,dobj,xx,True,True
0,.,.,PUNCT,.,punct,.,False,False
0,,,SPACE,,,,False,False
0,dan,dan,NOUN,NN,nsubj,xxx,True,False
0,does,do,VERB,VBZ,aux,xxxx,True,True


In [11]:
for token in parsed_review:
    print(token, token.sentiment)

horrible 0.0
class 0.0
. 0.0
DONT 0.0
take 0.0
it 0.0
. 0.0
  0.0
dan 0.0
does 0.0
n't 0.0
talk 0.0
about 0.0
anything 0.0
real 0.0
during 0.0
class 0.0
, 0.0
just 0.0
rambles 0.0
on 0.0
about 0.0
his 0.0
opinion 0.0
on 0.0
stuff 0.0
and 0.0
then 0.0
will 0.0
give 0.0
you 0.0
random 0.0
facts 0.0
to 0.0
write 0.0
down 0.0
so 0.0
you 0.0
can 0.0
write 0.0
the 0.0
essays 0.0
well 0.0
. 0.0
paper 0.0
due 0.0
every 0.0
week 0.0
, 0.0
do 0.0
all 0.0
of 0.0
them 0.0
. 0.0
and 0.0
i 0.0
d 0.0
write 0.0
all 0.0
the 0.0
essays 0.0
so 0.0
that 0.0
you 0.0
can 0.0
get 0.0
a 0.0
good 0.0
grade 0.0
. 0.0
go 0.0
to 0.0
class 0.0
because 0.0
of 0.0
random 0.0
quizzes 0.0
. 0.0
